## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [79]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable

groq_api_key=os.getenv("GROQ_API_KEY")

In [80]:
from langchain_groq import ChatGroq
model=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002DFFF6AFA60>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002DFFF6ADCF0>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [81]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi , My name is Omkar and I am a Machine Learning Engineer")])

AIMessage(content="Hi Omkar, it's nice to meet you! \n\nAs a Machine Learning Engineer, what are you currently working on? Do you have a particular area of ML that you specialize in?  \n\nI'm always interested in learning more about what people are doing in the field. 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 63, 'prompt_tokens': 23, 'total_tokens': 86, 'completion_time': 0.114545455, 'prompt_time': 0.002130875, 'queue_time': 0.090450058, 'total_time': 0.11667633}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-649f6c66-0077-4a6f-a523-904f75a32776-0', usage_metadata={'input_tokens': 23, 'output_tokens': 63, 'total_tokens': 86})

In [82]:
from langchain_core.messages import AIMessage
model.invoke([
    HumanMessage(content="Hi, My name is Omkar I am a Machine Learning Engineer"),
    AIMessage(content="Hello Omkar, It's nice to meet you. \n\n As a Machine Learning Engineer, what kind of projects are you working on?"),
    HumanMessage(content="Hey, What's my name and what do I do?")
])

AIMessage(content="You said your name is Omkar and you are a Machine Learning Engineer. 😊  \n\nIs there anything else you'd like to talk about? Perhaps you have a project you'd like to discuss or a question about machine learning?  I'm here to help!\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 74, 'total_tokens': 133, 'completion_time': 0.107272727, 'prompt_time': 0.00388912, 'queue_time': 0.066029773, 'total_time': 0.111161847}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1d2d1a56-d432-4463-af88-7950a5ef06f9-0', usage_metadata={'input_tokens': 74, 'output_tokens': 59, 'total_tokens': 133})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [83]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={} #store to keep the session_id distinguised from other sessions
def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history) #adding the message history to the model

In [84]:
config={"configurable":{"session_id":"chat1"}}

In [85]:
response=with_message_history.invoke([
    HumanMessage(content="Hi, My name is Omkar and I am a Machine Learning Engineer")]
    ,config=config)

In [86]:
response.content

"Hi Omkar, it's nice to meet you! \n\nAs a large language model, I'm always interested in connecting with people in the field of machine learning. \n\nWhat kind of work are you doing as a Machine Learning Engineer?  \n\nI'm happy to chat about anything related to ML, or just have a general conversation.\n"

In [87]:
with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

AIMessage(content='Your name is Omkar.  \n\nI remember that from our introduction. 😊  Is there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 112, 'total_tokens': 142, 'completion_time': 0.054545455, 'prompt_time': 0.005382127, 'queue_time': 0.092062167, 'total_time': 0.059927582}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-1fedcefc-c69a-42bb-81c3-62ef921d35a5-0', usage_metadata={'input_tokens': 112, 'output_tokens': 30, 'total_tokens': 142})

In [88]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

"As an AI, I have no memory of past conversations and do not know your name. If you'd like to tell me, I'm happy to know! 😊  \n\n"

In [89]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey, My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you! 👋 \n\nWhat can I do for you today? 😄\n"

In [90]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John,  😊  \n\nI remember! \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [92]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [93]:
chain.invoke({"messages":[HumanMessage(content="Hi, My name is Omkar")]})

AIMessage(content="Hi Omkar, it's nice to meet you! How can I help you today?  \n\nI'll do my best to answer your questions. Just ask away! 😊 \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 33, 'total_tokens': 75, 'completion_time': 0.076363636, 'prompt_time': 0.003606042, 'queue_time': 0.111218949, 'total_time': 0.079969678}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-67f16dc3-9c6b-4f6f-b38d-d11ac859f29a-0', usage_metadata={'input_tokens': 33, 'output_tokens': 42, 'total_tokens': 75})

In [94]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [95]:
config2 = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi, My name is Omkar")],
    config=config2
)

response.content

"Hello Omkar! It's nice to meet you.  \n\nI'm ready to answer your questions to the best of my ability. What can I help you with today? 😊  \n\n"

In [96]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Omkar.  \n\nI keep track of our conversations, so I remember! 😄  \n\nDo you have any other questions for me?\n'

In [121]:
## Add more complexity
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [122]:
response=chain.invoke({"messages":[HumanMessage(content="Hi, My name is Omkar")],"language":"Telugu"})
response.content

'నేను మీకు సహాయకరమైన సహచరునిగా ఉన్నాను. మీ ప్రశ్నలకు నేను నా సామర్థ్యం ఉన్నంత వరకు తెలుగులో గానీ, అంతిమ ప్రాధాన్యతతో తెలుగులోనే గానీ సమాధానం ఇస్తాను. \n\n"Omkar", మీ పేరు చాలా బాగుంది.  మీకు ఏమి సహాయం చేయాలి? \n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [123]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages" #input message key means the key in the input dictionary that contains the messages
)

In [156]:
config3 = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi, I am Omkar")],"language":"Hindi"},
    config=config3
)
repsonse.content

'नमस्ते ओमकार! \n\nआप कैसे हैं? 😊  मुझसे आप क्या पूछना चाहेंगे? \n'

In [157]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config3,
)
response.content

'आपका नाम ओमकार है।  🙂 \n'

In [158]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "telugu"},
    config=config3,
)
response.content

'మీ పేరు ఒంకార్. 😊 \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [105]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=55,
    strategy='last', #trim the last message
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on='human'
)

messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]

trimmer.invoke(messages)

[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content="hi! I'm bob", additional_kwargs={}, response_metadata={}),
 AIMessage(content='hi!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

See the conversation stored from only "I like vanilla icecream" not from "hi! I'm bob",First HumanMessage doesn't get stored looks like, so if you increase max_tokens it stores more info based on number given. Coz there is a limitation for every model's context info

In [106]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain_2=(
    RunnablePassthrough.assign(messages=itemgetter('messages')|trimmer)
    | prompt | model
)

response=chain_2.invoke({"messages":messages + [HumanMessage(content="What Icecream do I like?")],'language':'English'})
response.content

'You said you like vanilla ice cream!  🍦  \n\nIs there anything else I can help you with?\n'

In [107]:
response = chain_2.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked "what is 2 + 2" 😊  \n\n\n\nDo you want to try another one?\n'

In [109]:
response = chain_2.invoke(
    {
        "messages": messages + [HumanMessage(content="what is my name?")],
        "language": "English",
    }
)
response.content

"As a large language model, I don't have access to past conversations or personal information about you. So I don't know your name.  Would you like to tell me? 😊  \n"

In [110]:
response = chain_2.invoke(
    {
        "messages": messages + [HumanMessage(content="My name is omkar!")],
        "language": "English",
    }
)
response.content

"That's a great name, Omkar!  Nice to meet you.  😊  \n\nIs there anything else I can help you with today?\n"

In [111]:
response = chain_2.invoke(
    {
        "messages": messages + [HumanMessage(content="what is my name?")],
        "language": "English",
    }
)
response.content

"As a helpful assistant, I don't have access to past conversations or personal information about you, including your name.  \n\nIf you'd like to tell me your name, I'd be happy to use it! 😊\n"

Even though you said your name explicitly it doesn't store your name

In [117]:
## Lets wrap this in the Message History
## Now it's new memory and new chat which doesn't have previous chat memory
## here chain is new chain we defined above
with_message_history = RunnableWithMessageHistory(
    chain_2,
    get_session_history,
    input_messages_key="messages",
)
config4={"configurable":{"session_id":"chat5"}}

In [118]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config4,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you. So, I don't know your name. 😊  \n\nWould you like to tell me?\n"

In [120]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "hindi",
    },
    config=config4,
)

response.content

'मुझे माफ़ करना, लेकिन मैं पिछली बातचीत को याद नहीं रखता हूँ। आप क्या गणित समस्या पूछना चाहते हैं, कृपया बताएँ ताकि मैं आपकी मदद कर सकूं। 😊 \n'

In [115]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="My Name is Omkar!")],
        "language": "English",
    },
    config=config4,
)

response.content

"It's nice to meet you, Omkar! 😊 \n\nDo you have any other questions I can help you with?\n"

In [116]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config4,
)

response.content

"As an AI, I don't have access to past conversations or any personal information about you, including your name.  Would you like to tell me your name? 😊  \n"

Even though you used with_message_history with get_session_history function it's not storing any of the info you previosly shared and also not storing the current new info, even it's not sharing what you mentioned in trimmer part